# Simple Deer - Grass model


In [ ]:
import utils

import matplotlib.pyplot as plt
import math

In [ ]:
# Test out the CompartmentalModel class
# With two populations: deer and grass

# Define the starting populations in a dictionary
deer_grass_populations = {"deer": [100], "grass": [3000]}

# Define some functions that are the components of the model

def deer_grows(state_dictionary, timestep):
    # Deer grows proportional to the amount of grass, grass carrying capacity is 3000
    
    new_deer = state_dictionary["deer"] * 1.5 * (state_dictionary["grass"]/3000)
    return {"deer": new_deer, "grass": 0}

def deer_dies(state_dictionary, timestep):
    # Deer declines 1.1 of population every year

    deer_change = -1.1 * state_dictionary["deer"]
    return {"deer": deer_change, "grass": 0}

def grass_grows(state_dictionary, timestep):
    # The grass grows logistically with a carrying capacity of 3000 and a rate of 0.8
    return {"deer": 0, "grass": 0.8 * state_dictionary["grass"] * (1 - state_dictionary["grass"] / 3000)}

def grass_dies(state_dictionary, timestep):
    # The grass dies at a rate proportional to the number of deer
    return {"deer": 0, "grass": -1.2 * state_dictionary["deer"] * state_dictionary["grass"] / 3000}

# Compartments are on a new line just for readability
deer_grass_model = utils.CompartmentalModel(start_states=deer_grass_populations, 
                                                 compartments=[deer_grows, grass_grows, deer_dies, grass_dies],
                                                 time_units="years")

# Run the model for 14 years
deer_grass_model.simulate(timesteps=90)

# Check that it worked
deer_grass_model.print_states()


In [ ]:
# Plot

# Grab each set of states
from turtle import color


all_states = deer_grass_model.get_states()
deer_states = all_states["deer"]
grass_states = all_states["grass"]

# Get the timestamps
timestamps = [i for i in range(len(deer_states))]

# Plot the deer and grass populations over time
# `scatter` plots points instead of a smooth line like `plot`
plt.scatter(timestamps, deer_states, label="Deer (start: 100)", color="tab:orange")
plt.scatter(timestamps, grass_states, label="Grass (start: 3000)", color="tab:green")

# Add some labels
plt.title("Simple Deer - Grass Model")
plt.xlabel(f"Time ({deer_grass_model.get_time_units()})")
plt.ylim(bottom=0)
plt.ylabel("Population (Deer: individuals, Grass: density units)")
plt.legend()
# uncomment the following line to control the figure size in inches (standard is 6.4, 4.8)
# plt.rcParams["figure.figsize"] = (6.4, 4.8)

plt.show()
plt.close()

In [ ]:
# Code to check when the deer population drops below 1
# Only works if this actually happens

# deer_under_1 = []
# for i, deer in enumerate(deer_states):
#     if deer < 1:
#         deer_under_1.append(i)

# len(deer_under_1), min(deer_under_1), max(deer_under_1)

In [ ]:
# Compare the linear term with Caughley's correction

# points for plotting
x = [i for i in range(5000)]

# linear term
y = [i / 3000 for i in x]
# Caughley's correction
z = [1 - math.exp(-0.001 *i) for i in x]

plt.plot(x, y, label="Linear")
plt.plot(x, z, label="Caughley", color="tab:red")
plt.vlines(3000, 0, 1.75, label="Carrying capacity", linestyle="dashed", color="tab:gray")

plt.legend()
plt.title("Linear vs Caughley grass-deer interaction")
plt.xlabel("Grass density")
plt.ylabel("Proportional change from eating")